In [23]:
import sys
from tqdm import tqdm
import pandas as pd

sys.path.append('/work/magroup/kaileyhu/synthetic_lethality/classifier/training/input_setup/')

from data_generator import data_generator

In [4]:
dg = data_generator(
        is_hvg = True,
        ko_file_dir = "/work/magroup/kaileyhu/res/perturbed/DUAL_BRCA_PARP/perturbed_embs/",
        sub_file_dir = "/work/magroup/kaileyhu/res/perturbed/DUAL_BRCA_PARP/subtracted_embs/",
        output_dir = "/work/magroup/kaileyhu/res/perturbed/DUAL_BRCA_PARP/generated_df",
        model_path = "/work/magroup/kaileyhu/Geneformer/gf-12L-30M-i2048/",
        model_type = "Pretrained",
        emb_mode = "cell",
        emb_dim = 512,
        n_classes = 0,
        dataset = "/work/magroup/kaileyhu/res/hvg_500_w_SL_tokenized_2048.dataset")

In [5]:
dg.remove_nan()

Extracting original embeddings from dataset /work/magroup/kaileyhu/res/hvg_500_w_SL_tokenized_2048.dataset


BertForMaskedLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.
/home/kaileyhu/anaconda3/envs/geneformer/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:774: UserWarning: `return_dict_in_generate` is N

  0%|          | 0/15 [00:00<?, ?it/s]

Removing nan values from original embedding
Original embedding has NAN values? False


In [6]:
dg.process_ko()

Processing knockout files, total: 160


100%|██████████| 160/160 [05:38<00:00,  2.12s/it]


In [7]:
dg.process_sub_no_via()

Processing subtracted files, total: 160


100%|██████████| 160/160 [05:29<00:00,  2.06s/it]


In [8]:
dg.remove_via_col()

Removing viability column from each dataframe...


100%|██████████| 160/160 [00:00<00:00, 90018.60it/s]


In [13]:
res_file = f"/work/magroup/kaileyhu/res/perturbed/DUAL_BRCA_PARP/generated_df/gene_patient_emb_mat.csv"

In [14]:
sub_embs = adata.uns['embedding_differences']

In [21]:
diff_dict = {}
for gene in tqdm(sub_embs):
    df_diff = sub_embs[gene]
    for patient in df_diff.index:
        split = gene.split(' ')
        
        # account for double gene names
        if (len(split) > 1):
            diff_dict[(patient, split[0], f"gene_{split[1]}")] = df_diff.loc[patient].tolist()

100%|██████████| 160/160 [00:14<00:00, 11.28it/s]


In [24]:
df = pd.DataFrame.from_dict(diff_dict, orient='index')
df.to_csv(res_file)